# Favorita: Favorita Grocery Sales Forecasting

## Data exploration  
For each data file, I want to see:  
(1) some samples with display(df.head(5)) or display(df.tail(5))  
(2) A summary of this DataFrame with df.describe()  
(3) Check if there is missing data with df.isnull().values.any()  
(4) The number of unique values for each variable with display(df['column_name'].unique())

In [1]:
# Import libraries necessary for this project
import os.path
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from IPython.display import display
import matplotlib.pyplot as plt

# Import supplementary visualizations code visuals.py
#import visuals as vs

# Pretty display for notebooks
%matplotlib inline


def loaddata(filename, nrows=None):
    types = {'id': 'int32', 'item_nbr': 'int32', 'store_nbr': 'int16', 'unit_sales': 'float32', 'onpromotion': bool,}
    data = pd.read_csv(filename, parse_dates=['date'], dtype=types, nrows=nrows, infer_datetime_format=True)
    return data

Training data:

In [2]:
# Load the training dataset
import sqlite3
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///data/favorita.db')

train_pkl = 'train_2017_8.csv'
years = range(2013, 2018)
months = range(1, 13)
if not os.path.isfile(train_pkl):
    train_data = loaddata('input/train.csv')
    print("Training dataset has {} data points with {} variables each.".format(*train_data.shape))
#    train_data['date'] = pd.to_datetime(train_data['date'], format='%Y-%m-%d')
#     train_data.to_sql('train', engine)
    
# save pickle file for each month
    for year in years:
#         for month in months:
#             file_name = 'train_' + str(year) + '_' + str(month) + '.pkl'
#             start = str(year) + '-' + str(month) + '-01'
#             if month<12:
#                 end = str(year) + '-' + str(month + 1) + '-01'
#             else:
#                 end = str(year + 1) + '-01-01'
        file_name = 'train_' + str(year) + '_8.csv'
        start = str(year) + '-08-16'
        end = str(year) + '-08-31'

        mask = (train_data['date']>=start)&(train_data['date']<=end)
        train_data.loc[mask].to_csv(file_name) 
  
# save pickle file for three even files
#     one_third_index = int(train_data.shape[0] // 3)
#     two_third_index = int(train_data.shape[0] * 2 // 3)
#     #pickle can only dump a file with maximum size 2GB, so the train.cvs (5GB) is saved into 3 files
#     pickle.dump(train_data.iloc[:one_third_index, :], open( "train1.pkl", "wb" ))
#     pickle.dump(train_data.iloc[one_third_index:two_third_index, :], open( "train2.pkl", "wb" ))
#     pickle.dump(train_data.iloc[two_third_index:, :], open( "train3.pkl", "wb" ))
else:
    train_data = pickle.load(open( train_pkl, "rb" ))
    train_data['date'] = pd.to_datetime(train_data['date'], format='%Y-%m-%d')
display(train_data.head(n=10))

/home/adou/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Training dataset has 125497040 data points with 6 variables each.


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN
5,5,2013-01-01,25,108786,3.0,NaN
6,6,2013-01-01,25,108797,1.0,NaN
7,7,2013-01-01,25,108952,1.0,NaN
8,8,2013-01-01,25,111397,13.0,NaN
9,9,2013-01-01,25,114790,3.0,NaN


In [ ]:
train_data.dtypes.value_counts()


### 1. Train.csv
Possilbe vales for each variable:
onpromotion: [nan, False, True] (onpromotion is for a specified date and store_nbr), appoximatily 16% ot eh onpromotion values in this file is NaN.  
train1.pkl (date range from 2013-01-01 to 2015-02-25)



In [ ]:
num_samples = train_data.shape[0]
display(train_data.head(n=1))
X_train_raw = train_data.drop(['unit_sales', 'id'], axis = 1, inplace = False)
print("Favorita grocery sales forecasting has {} samples with {} features each.".format(*X_train_raw.shape))
X_train = X_train_raw.iloc[:num_samples, :]
X_train.sort_values(['date'], ascending=True)
display(X_train.head(5))
display(X_train.tail(5))



In [ ]:
train_2014 = pickle.load(open('train_2014.pkl', 'rb'))
# train_2014.loc[(train_2014['onpromotion'] == True) | (train_2014['onpromotion'] == False)]
train_2014.isnull().values.any()

### 2. holidays_events.csv  
1. data:
2. type: ['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event']
3. locale: ['Local', 'Regional', 'National']
4. locale_name: ['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo','Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo','El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato', 'Ibarra','Quevedo', 'Santo Domingo de los Tsachilas', 'Santa Elena', 'Quito','Loja', 'Salinas', 'Guayaquil']
5. description:
6. transferred: [False,  True]

In [ ]:
holidays_events = pd.read_csv("input/holidays_events.csv")
display(holidays_events.describe())
display(holidays_events.head(n=1))
#display(holidays_events['transferred'].unique())
holidays_events.isnull().values.any()

### 3. stores.csv  
1. store_nbr: [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
2. city: ['Quito', 'Santo Domingo', 'Cayambe', 'Latacunga', 'Riobamba', 'Ibarra', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil', 'Salinas', 'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad', 'Cuenca', 'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen']
3. state: ['Pichincha', 'Santo Domingo de los Tsachilas', 'Cotopaxi', 'Chimborazo', 'Imbabura', 'Bolivar', 'Pastaza', 'Tungurahua', 'Guayas', 'Santa Elena', 'Los Rios', 'Azuay', 'Loja', 'El Oro', 'Esmeraldas', 'Manabi']
4. type: ['D', 'B', 'C', 'E', 'A']
5. cluster: [13,  8,  9,  4,  6, 15,  7,  3, 12, 16,  1, 10,  2,  5, 11, 14, 17]

In [ ]:
stores = pd.read_csv("input/stores.csv")
display(stores.describe())
display(stores.head(n=1))
display(stores['cluster'].unique())

### 4. oil.csv
dcoilwtico: continuous value from 26.19~110.62

In [ ]:
oil = pd.read_csv("input/oil.csv")
oil['date'] = pd.to_datetime(oil['date'], format='%Y-%m-%d')
display(oil.describe())
# display(oil.head(n=5))
train_data= pd.merge(train_data,oil, right_on='date',left_on='date',how='left')
display(train_data.tail(5))

### 5. transactions.csv

In [ ]:
import random
stores_nbr = random.sample(range(1,54), 5)
transactions = pd.read_csv("input/transactions.csv")
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')
display(transactions.describe())
display(transactions.head(n=1))
train_data= pd.merge(train_data,transactions, left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr'], how='left')
display(train_data.tail(5))

# for i_store in range(len(stores_nbr)):
#     fig = plt.figure(figsize=(10, 15))
#     for i in range(len(years)):
#         start = years[i] + '-01-01'
#         end = years[i] + '-12-31'
#         mask = (transactions['date']>start)&(transactions['date']<=end) & (transactions['store_nbr'] == stores_nbr[i_store])
#         ax = fig.add_subplot(1, len(years), i+1)
#         #plt.figure(i_store)
#         specific_store_year = transactions['transactions'][mask]
#         plt.plot(range(len(specific_store_year)), specific_store_year)
# plt.show()

### 6. items.csv
1. item_nbr: 4100 discrete values
2. class: 337 discrete values
3. perishable: continuous

In [ ]:
items = pd.read_csv("input/items.csv")
display(items.describe())
display(items.head(n=5))
display(items.sample(6))

train_data= pd.merge(train_data,items, right_on='item_nbr',left_on='item_nbr',how='left')
display(train_data.tail(5))

### test.csv

In [ ]:
test_data = pd.read_csv("input/test.csv")
print("Favorita grocery sales forecasting testing data has {} samples with {} features each.".format(*test_data.shape))
test_data['date'] = pd.to_datetime(test_data['date'])
test_data.sort_values('date', ascending=True)
display(test_data.head(5))
display(test_data.tail(5))

### sample_submission.csv

In [ ]:
sample_submission = pd.read_csv("input/sample_submission.csv")
display(sample_submission.head(n=5))
sample_submission.dtypes